In [1]:
import shap
import pandas as pd
import pickle
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import gridspec

In [26]:
from alibi.explainers import CounterfactualProto
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

tf.compat.v1.disable_eager_execution()
tf.keras.backend.clear_session()

In [2]:
train_full = pd.read_csv("./data/base/train-white.csv", sep=",")
test_full = pd.read_csv("./data/base/test-white.csv", sep=",")

In [3]:
target = "quality"
model_name = "rf"

In [4]:
with open('./model/selected-{}.pickle'.format(model_name), 'rb') as f:
    model = pickle.load(f)

In [5]:
X_train = copy.deepcopy(train_full.drop(columns=target))
X_test = copy.deepcopy(test_full.drop(columns=target))
y_train = train_full[target]
y_test = test_full[target]

In [55]:
X_train.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,3428.000000,3428.000000,3428.000000,3428.000000,3428.000000,3428.000000,3428.000000,3428.000000,3428.000000,3428.000000,3428.000000
mean,6.845741,0.280588,0.332211,6.429813,0.045718,35.246791,138.632439,0.994039,3.189454,0.491085,10.512968
std,0.843013,0.102237,0.120013,5.004096,0.021655,17.106975,42.193279,0.002914,0.149611,0.114747,1.232832
min,3.900000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987130,2.720000,0.230000,8.400000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,109.000000,0.991780,3.090000,0.410000,9.475000
50%,6.800000,0.260000,0.310000,5.300000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000
75%,7.300000,0.330000,0.382500,9.900000,0.050000,46.000000,167.000000,0.996120,3.280000,0.550000,11.400000
max,14.200000,1.100000,1.230000,31.600000,0.346000,289.000000,440.000000,1.010300,3.820000,1.010000,14.200000


In [59]:
shape = (1,) + X_train.shape[1:]

In [60]:
explainer = CounterfactualProto(model.predict_proba, shape, use_kdtree=True, theta=10., feature_range=(0, 1000))

In [62]:
explainer.fit(X_train.to_numpy(), trustscore_kwargs=None)

No encoder specified. Using k-d trees to represent class prototypes.


CounterfactualProto(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.1,
              'gamma': 0.0,
              'theta': 10.0,
              'cat_vars': None,
              'ohe': False,
              'use_kdtree': True,
              'learning_rate_init': 0.01,
              'max_iterations': 1000,
              'c_init': 10.0,
              'c_steps': 10,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'feature_range': (0, 1000),
              'shape': (1, 11),
              'is_model': False,
              'is_ae': False,
              'is_enc': False,
              'enc_or_kdtree': True,
              'is_cat': False,
              'trustscore_kwargs': None,
              'd_type': 'abdm',
              'w': None,
              'disc_perc

In [66]:
explanations = explainer.explain(X_test.iloc[0].to_numpy().reshape(1, -1), k=2)

In [54]:
explanations

Explanation(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.1,
              'gamma': 0.0,
              'theta': 10.0,
              'cat_vars': None,
              'ohe': False,
              'use_kdtree': True,
              'learning_rate_init': 0.01,
              'max_iterations': 1000,
              'c_init': 10.0,
              'c_steps': 10,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'feature_range': (-6, 12),
              'shape': (1, 11),
              'is_model': False,
              'is_ae': False,
              'is_enc': False,
              'enc_or_kdtree': True,
              'is_cat': False,
              'trustscore_kwargs': None,
              'd_type': 'abdm',
              'w': None,
              'disc_perc': (25, 5

In [34]:
explanations2 = explainer.explain(X_test.iloc[1].to_numpy().reshape(1, -1), k=2)

In [35]:
explanations2

Explanation(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.1,
              'gamma': 0.0,
              'theta': 10.0,
              'cat_vars': None,
              'ohe': False,
              'use_kdtree': True,
              'learning_rate_init': 0.01,
              'max_iterations': 1000,
              'c_init': 10.0,
              'c_steps': 10,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'feature_range': (-10, 10),
              'shape': (1, 11),
              'is_model': False,
              'is_ae': False,
              'is_enc': False,
              'enc_or_kdtree': True,
              'is_cat': False,
              'trustscore_kwargs': None,
              'd_type': 'abdm',
              'w': None,
              'disc_perc': (25, 